# 机器人实例创建

In [1]:
import asyncio
import lebai_sdk

In [2]:
print(await lebai_sdk.discover_devices(2))

[{'name': 'lebai-LLlzqh', 'mac': 'b4:4b:d6:5e:55:8c', 'ip': '192.168.0.50', 'online': True}]


In [3]:
robot_ip = "192.168.0.50" #设定机器人ip地址，需要根据机器人实际ip地址修改
lebai = await lebai_sdk.connect(robot_ip, False) #创建实例
#True是仿真模式
print(lebai)

In [4]:
await lebai.start_sys() #启动手臂

In [5]:
# await lebai.stop_sys()

# 服务端处理

# 初始化函数 

In [6]:
#归位官方初始位置
async def return_pose():
    print('正在开始初始化……')
    await lebai.init_claw() #初始化夹爪
    await lebai.start_sys() #启动手臂
    joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    a = 0.5 #关节加速度 (rad/s2)
    v = 0.2 #关节速度 (rad/s)
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movej(joint_pose,a,v,t,r)
    a = 0.3 #空间加速度 (m/s2)
    v = 0.1 #空间速度 （m/s）
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    await lebai.movel(cartesian_pose,a,v,t,r) #直线运动 https://help.lebai.ltd/sdk/motion.html#%E7%9B%B4%E7%BA%BF%E8%BF%90%E5%8A%A8
    lebai.wait_move() #等待运动完成
    print('初始化归位完成')

In [7]:
await return_pose()

正在开始初始化……
初始化归位完成


In [8]:
#def start_ros():
    

In [9]:
#def stop_ros():

# 移动函数

In [10]:
#x+移动
async def x_right():
    print("往x+移动")
    data=await lebai.get_kin_data()
    #print(data)
    #print(data['actual_joint_pose'])
    # joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    # cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    print("法兰盘坐标：")
    dict_flange=data['actual_flange_pose']['position']
    
    print("欧拉角：")
    dict_euler_now=data['actual_flange_pose']['rotation']['euler_zyx']
    print(dict_euler_now)
    #print(type(k))
    keys=['rz','ry','rx']
    values=[dict_euler_now['z'],dict_euler_now['y'],dict_euler_now['x']]
    dict_euler_target=dict(zip(keys,values))
    print(dict_euler_target)
    cartesian_pose={**dict_flange,**dict_euler_target}
    #print(cartesian_pose)
    #print(cartesian_pose['x'])
    x=cartesian_pose['x']
    print(x)
    if(x+0.05<1):
        cartesian_pose['x']=x+0.05
    elif(x+0.05>1):
        print("超出边界")
    
    a = 0.3 #空间加速度 (m/s2)
    v = 0.1 #空间速度 （m/s）
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    await lebai.movel(cartesian_pose,a,v,t,r) #直线运动 https://help.lebai.ltd/sdk/motion.html#%E7%9B%B4%E7%BA%BF%E8%BF%90%E5%8A%A8
    lebai.wait_move() #等待运动完成
    print("x+移动完成")
    

In [11]:
##x-移动
async def x_left():
    print("往x-移动")
    data=await lebai.get_kin_data()
    #print(data)
    #print(data['actual_joint_pose'])
    # joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    # cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    print("法兰盘坐标：")
    dict_flange=data['actual_flange_pose']['position']
    
    print("欧拉角：")
    dict_euler_now=data['actual_flange_pose']['rotation']['euler_zyx']
    print(dict_euler_now)
    #print(type(k))
    keys=['rz','ry','rx']
    values=[dict_euler_now['z'],dict_euler_now['y'],dict_euler_now['x']]
    dict_euler_target=dict(zip(keys,values))
    print(dict_euler_target)
    cartesian_pose={**dict_flange,**dict_euler_target}
    #print(cartesian_pose)
    #print(cartesian_pose['x'])
    x=cartesian_pose['x']
    #边界检测
    print(x)
    if(x-0.05>-2):
        cartesian_pose['x']=x-0.05
    elif(x-0.05<-2):
        print("超出边界")
    
    a = 0.3 #空间加速度 (m/s2)
    v = 0.1 #空间速度 （m/s）
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movel(cartesian_pose,a,v,t,r) #直线运动 https://help.lebai.ltd/sdk/motion.html#%E7%9B%B4%E7%BA%BF%E8%BF%90%E5%8A%A8
    lebai.wait_move() #等待运动完成
    print("x-移动完成")
    

In [12]:
#y+移动
async def y_right():
    print("往y+移动")
    data=await lebai.get_kin_data()
    #print(data)
    #print(data['actual_joint_pose'])
    # joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    # cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    print("法兰盘坐标：")
    dict_flange=data['actual_flange_pose']['position']
    
    print("欧拉角：")
    dict_euler_now=data['actual_flange_pose']['rotation']['euler_zyx']
    print(dict_euler_now)
    #print(type(k))
    keys=['rz','ry','rx']
    values=[dict_euler_now['z'],dict_euler_now['y'],dict_euler_now['x']]
    dict_euler_target=dict(zip(keys,values))
    print(dict_euler_target)
    cartesian_pose={**dict_flange,**dict_euler_target}
    #print(cartesian_pose)
    #print(cartesian_pose['x'])
    y=cartesian_pose['y']
    print(y)
    if(y+0.05<2):
        cartesian_pose['y']=y+0.05
    elif(y+0.05>2):
        print("超出边界")
    
    a = 0.3 #空间加速度 (m/s2)
    v = 0.1 #空间速度 （m/s）
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movel(cartesian_pose,a,v,t,r) #直线运动 https://help.lebai.ltd/sdk/motion.html#%E7%9B%B4%E7%BA%BF%E8%BF%90%E5%8A%A8
    lebai.wait_move() #等待运动完成
    print("y+移动完成")

In [13]:
#y-移动
async def y_left():
    print("往y-移动")
    data=await lebai.get_kin_data()
    #print(data)
    #print(data['actual_joint_pose'])
    # joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    # cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    print("法兰盘坐标：")
    dict_flange=data['actual_flange_pose']['position']
    
    print("欧拉角：")
    dict_euler_now=data['actual_flange_pose']['rotation']['euler_zyx']
    print(dict_euler_now)
    #print(type(k))
    keys=['rz','ry','rx']
    values=[dict_euler_now['z'],dict_euler_now['y'],dict_euler_now['x']]
    dict_euler_target=dict(zip(keys,values))
    print(dict_euler_target)
    cartesian_pose={**dict_flange,**dict_euler_target}
    #print(cartesian_pose)
    #print(cartesian_pose['x'])
    y=cartesian_pose['y']
    print(y)
    if(y-0.05>-2):
        cartesian_pose['y']=y-0.05
    elif(y-0.05<-2):
        print("超出边界")
    
    a = 0.3 #空间加速度 (m/s2)
    v = 0.1 #空间速度 （m/s）
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movel(cartesian_pose,a,v,t,r) #直线运动 https://help.lebai.ltd/sdk/motion.html#%E7%9B%B4%E7%BA%BF%E8%BF%90%E5%8A%A8
    lebai.wait_move() #等待运动完成
    print("y-移动完成")

In [ ]:
#z-移动
async def z_left():
    print("往y-移动")
    data=await lebai.get_kin_data()
    #print(data)
    #print(data['actual_joint_pose'])
    # joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    # cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    print("法兰盘坐标：")
    dict_flange=data['actual_flange_pose']['position']
    
    print("欧拉角：")
    dict_euler_now=data['actual_flange_pose']['rotation']['euler_zyx']
    print(dict_euler_now)
    #print(type(k))
    keys=['rz','ry','rx']
    values=[dict_euler_now['z'],dict_euler_now['y'],dict_euler_now['x']]
    dict_euler_target=dict(zip(keys,values))
    print(dict_euler_target)
    cartesian_pose={**dict_flange,**dict_euler_target}
    #print(cartesian_pose)
    #print(cartesian_pose['x'])
    y=cartesian_pose['y']
    print(y)
    if(z-0.05>-2):
        cartesian_pose['y']=z-0.05
    elif(z-0.05<-2):
        print("超出边界")
    
    a = 0.3 #空间加速度 (m/s2)
    v = 0.1 #空间速度 （m/s）
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movel(cartesian_pose,a,v,t,r) #直线运动 https://help.lebai.ltd/sdk/motion.html#%E7%9B%B4%E7%BA%BF%E8%BF%90%E5%8A%A8
    lebai.wait_move() #等待运动完成
    print("y-移动完成")

In [ ]:
#z+移动
async def z_right():
    print("往y-移动")
    data=await lebai.get_kin_data()
    #print(data)
    #print(data['actual_joint_pose'])
    # joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    # cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    print("法兰盘坐标：")
    dict_flange=data['actual_flange_pose']['position']
    
    print("欧拉角：")
    dict_euler_now=data['actual_flange_pose']['rotation']['euler_zyx']
    print(dict_euler_now)
    #print(type(k))
    keys=['rz','ry','rx']
    values=[dict_euler_now['z'],dict_euler_now['y'],dict_euler_now['x']]
    dict_euler_target=dict(zip(keys,values))
    print(dict_euler_target)
    cartesian_pose={**dict_flange,**dict_euler_target}
    #print(cartesian_pose)
    #print(cartesian_pose['x'])
    z=cartesian_pose['z']
    print(z)
    if(z-0.05>-2):
        cartesian_pose['z']=z-0.05
    elif(z-0.05<-2):
        print("超出边界")
    
    a = 0.3 #空间加速度 (m/s2)
    v = 0.1 #空间速度 （m/s）
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movel(cartesian_pose,a,v,t,r) #直线运动 https://help.lebai.ltd/sdk/motion.html#%E7%9B%B4%E7%BA%BF%E8%BF%90%E5%8A%A8
    lebai.wait_move() #等待运动完成
    print("z-移动完成")

In [14]:
#await x_right()

In [15]:
# await y_right()

In [16]:
async def z_down():
    print("往z-移动")
    joint_pose = [-0.037870150700926766, 0.32549154842948447, 0.6361226579763268, -0.8806967198448438, 1.6284164801398509, -0.0011504855909142309] #目标位姿关节数据
    cartesian_pose = {'x' : -0.493, 'y' :-0.099, 'z' : -0.196, 'rz' : -1.66, 'ry' : 0.005, 'rx' : 1.65}#目标位姿笛卡尔数据
    a = 0.5 #关节加速度 (rad/s2)
    v = 0.2 #关节速度 (rad/s)
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movej(joint_pose,a,v,t,r)
    time.sleep(10)
    #lebai.set_claw(0,100)

In [17]:
async def z_up():
    print("往z+移动")
    # lebai.set_claw(40,20)
    joint_pose = [0,-1.05,1.05,0,1.57,0] #目标位姿关节数据
    cartesian_pose = {'x' : -0.383, 'y' :-0.121, 'z' : 0.36, 'rz' : -1.57, 'ry' : 0, 'rx' : 1.57}#目标位姿笛卡尔数据
    a = 0.5 #关节加速度 (rad/s2)
    v = 0.2 #关节速度 (rad/s)
    t = 0   #运动时间 (s)。 当 t > 0 时，参数速度 v 和加速度 a 无效
    r = 0   #交融半径 (m)。用于指定路径的平滑效果
    lebai.movej(joint_pose,a,v,t,r) 

# 夹爪函数

In [18]:
#松开函数
async def release():
    lebai.set_claw(0,100)
    print("夹爪松开")

In [19]:
#夹紧函数
async def clamp():
    lebai.set_claw(40,20)
    print("夹爪夹紧")

# 主函数

In [20]:
import socket
import asyncio
import lebai_sdk
import time
print(await lebai_sdk.discover_devices(2))
robot_ip = "192.168.0.50" #设定机器人ip地址，需要根据机器人实际ip地址修改
lebai = await lebai_sdk.connect(robot_ip, False) #创建实例
#True是仿真模式
print(lebai)
await lebai.start_sys() #启动手臂


language={'a':'x-','d':'x+','q':'y-','e':'y+','w':'z+','s':'z-','o':'o','p':'p','r':'r','k':'start','l':'stop'}
#  a表示x-方向,d表示x+方向，w表示z+方向，s表示z-方向，
#  o表示张开，p表示闭合
#  r表示return初始化归位
host='127.0.0.1'
port=5673
s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)#声明socket类型面向连接
s.bind((host,port))
s.listen(3)
print("正在监听{}端口".format(port))
conn,addr=s.accept()
print("连接的地址和端口：{}".format(addr))#打印监听的地址

#初始化
#await return_pose()

while True:
    data = conn.recv(2048)
    data=data.decode()
    if not data:
        break
    print("接收到的数据为：{}".format(data))#打印接受到的数据
    if(data=='s'):#z放下
        await z_down()
    elif(data=='w'):#z夹上
        await z_up()
    elif(data=='q'):#y-
        await y_left()
    elif(data=='e'): #y+
        await y_right()
    elif(data=='a'):# x-
        await x_left()
    elif(data=='d'):# x+
        await x_right()
    elif(data=='o'):#释放
        await release()
    elif(data=='p'):#夹紧
        await clamp()
    elif(data=='r'):#归位
        await return_pose()
    elif(data=='k'):#启动
        await lebai.start_sys()
    elif(data=='l'):#关闭
        await lebai.stop_sys()        
    conn.sendall(language.get(data,'Nothing').encode())
conn.close()
s.close()

[{'name': 'lebai-LLlzqh', 'mac': 'b4:4b:d6:5e:55:8c', 'ip': '192.168.0.50', 'online': True}]


OSError: [Errno 98] Address already in use